In [1]:
## import statements ##

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score

In [25]:
## data loading ##

## load data ##

data_rain = pd.read_csv('data_mrc/Rainfall_StungTreng.csv')
data_flow = pd.read_csv('data_mrc/Discharge_StungTreng.csv')
data_gdp = pd.read_csv('data_imf/thailand_gdp_imf.csv', delimiter=';')

data_rain = data_rain.rename(columns={'Value': 'rainfall'})
data_flow = data_flow.rename(columns={'Value': 'flowrate'})
data_gdp = data_gdp.rename(columns={'GDP (Billions of U.S. dollars)': 'gdp'})

data_rain = data_rain[['Timestamp (UTC+07:00)', 'rainfall']]
data_flow = data_flow[['Timestamp (UTC+07:00)', 'flowrate']]

# merge data
df = pd.merge(data_rain, data_flow, on='Timestamp (UTC+07:00)', how='inner')
df['Timestamp (UTC+07:00)'] = pd.to_datetime(df['Timestamp (UTC+07:00)'], format='%Y-%m-%d %H:%M:%S')
df = df.rename(columns={'Timestamp (UTC+07:00)': 'date'})
df.set_index('date', inplace=True)

# Calculate the minimum flowrate over the past week / month (base flowrate)
df['min_flow_week'] = df['flowrate'].rolling(window=7, min_periods=1).min()
df['min_flow_month'] = df['flowrate'].rolling(window=30, min_periods=1).min()

# add lag rainfall features
df['rainfall_lag1'] = df['rainfall'].shift(1)
df['rainfall_lag2'] = df['rainfall'].shift(2)
df['rainfall_lag3'] = df['rainfall'].shift(3)
df['rainfall_lag5weeks'] = df['rainfall'].shift(35)
df['rainfall_week'] = df['rainfall'].rolling(window=7, min_periods=1).mean()
df['rainfall_total_week'] = df['rainfall'].rolling(window=7, min_periods=1).sum()

# extract DateTime components
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df['day_of_week'] = df.index.dayofweek
df['day_of_year'] = df.index.dayofyear
df['week_of_year'] = df.index.isocalendar().week.astype(int)

# merge with gdp data
df = pd.merge(df, data_gdp, on='year', how='left')

# separate data by month
df_january = df[df['month'] == 1]
df_february = df[df['month'] == 2]
df_march = df[df['month'] == 3]
df_april = df[df['month'] == 4]
df_may = df[df['month'] == 5]
df_june = df[df['month'] == 6]
df_july = df[df['month'] == 7]
df_august = df[df['month'] == 8]
df_september = df[df['month'] == 9]
df_october = df[df['month'] == 10]
df_november = df[df['month'] == 11]
df_december = df[df['month'] == 12]

In [26]:
# drop non-needed columns
df = df.drop(columns=['year', 'day', 'day_of_week', 'day_of_year', 'rainfall_week'])
df_january = df_january.drop(columns=['year', 'day', 'day_of_week', 'day_of_year', 'rainfall_week'])
df_february = df_february.drop(columns=['year', 'day', 'day_of_week', 'day_of_year', 'rainfall_week'])
df_march = df_march.drop(columns=['year', 'day', 'day_of_week', 'day_of_year', 'rainfall_week'])
df_april = df_april.drop(columns=['year', 'day', 'day_of_week', 'day_of_year', 'rainfall_week'])
df_may = df_may.drop(columns=['year', 'day', 'day_of_week', 'day_of_year', 'rainfall_week'])
df_june = df_june.drop(columns=['year', 'day', 'day_of_week', 'day_of_year', 'rainfall_week'])
df_july = df_july.drop(columns=['year', 'day', 'day_of_week', 'day_of_year', 'rainfall_week'])
df_august = df_august.drop(columns=['year', 'day', 'day_of_week', 'day_of_year', 'rainfall_week'])
df_september = df_september.drop(columns=['year', 'day', 'day_of_week', 'day_of_year', 'rainfall_week'])
df_october = df_october.drop(columns=['year', 'day', 'day_of_week', 'day_of_year', 'rainfall_week'])
df_november = df_november.drop(columns=['year', 'day', 'day_of_week', 'day_of_year', 'rainfall_week'])
df_december = df_december.drop(columns=['year', 'day', 'day_of_week', 'day_of_year', 'rainfall_week'])

# drop rows with missing values
df_january.dropna(inplace=True)
df_february.dropna(inplace=True)
df_march.dropna(inplace=True)
df_april.dropna(inplace=True)
df_may.dropna(inplace=True)
df_june.dropna(inplace=True)
df_july.dropna(inplace=True)
df_august.dropna(inplace=True)
df_september.dropna(inplace=True)
df_october.dropna(inplace=True)
df_november.dropna(inplace=True)
df_december.dropna(inplace=True)

In [27]:
df_march.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 381 entries, 160 to 4656
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rainfall             381 non-null    float64
 1   flowrate             381 non-null    float64
 2   min_flow_week        381 non-null    float64
 3   min_flow_month       381 non-null    float64
 4   rainfall_lag1        381 non-null    float64
 5   rainfall_lag2        381 non-null    float64
 6   rainfall_lag3        381 non-null    float64
 7   rainfall_lag5weeks   381 non-null    float64
 8   rainfall_total_week  381 non-null    float64
 9   month                381 non-null    int64  
 10  week_of_year         381 non-null    int32  
 11  gdp                  381 non-null    float64
dtypes: float64(10), int32(1), int64(1)
memory usage: 37.2 KB


In [28]:
## Scaling function

def scale_data(data):
    scaler_rain = StandardScaler()
    data[['rainfall', 'rainfall_lag1','rainfall_lag2','rainfall_lag3', 'rainfall_lag5weeks' , 'rainfall_total_week']] = scaler_rain.fit_transform(data[['rainfall', 'rainfall_lag1','rainfall_lag2','rainfall_lag3', 'rainfall_lag5weeks' , 'rainfall_total_week']])
    scaler_flow = StandardScaler()
    data[['flowrate']] = scaler_flow.fit_transform(data[['flowrate']])
    scaler_statflow = StandardScaler()
    data[['min_flow_week','min_flow_month']] = scaler_statflow.fit_transform(data[['min_flow_week','min_flow_month']])
    scaler_gdp = StandardScaler()
    data[['gdp']] = scaler_gdp.fit_transform(data[['gdp']])
    return data

df_january = scale_data(df_january)
df_february = scale_data(df_february)
df_march = scale_data(df_march)
df_april = scale_data(df_april)
df_may = scale_data(df_may)
df_june = scale_data(df_june)
df_july = scale_data(df_july)
df_august = scale_data(df_august)
df_september = scale_data(df_september)
df_october = scale_data(df_october)
df_november = scale_data(df_november)
df_december = scale_data(df_december)


In [29]:
## all data - monthly ##

def linreg(data):
    X = data[['rainfall', 'week_of_year', 'rainfall_lag1', 'rainfall_lag2', 'rainfall_lag3', 'rainfall_lag5weeks' ,'rainfall_total_week', 'min_flow_week', 'min_flow_month']]
    y = data['flowrate']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train = X_train.sort_index()
    y_train = y_train.sort_index()
    X_test = X_test.sort_index()
    y_test = y_test.sort_index()

    lr_model = LinearRegression()

    # Perform 5-fold cross-validation
    cv_scores_mse = cross_val_score(lr_model, X, y, cv=5, scoring='neg_mean_squared_error')
    cv_scores_r2 = cross_val_score(lr_model, X, y, cv=5, scoring='r2')

    #mean_cv_mse = -cv_scores_mse.mean()
    #mean_cv_r2 = cv_scores_r2.mean()

    lr_model.fit(X_train, y_train)
    lr_pred = lr_model.predict(X_test)

    lr_mse = mean_squared_error(y_test, lr_pred)
    lr_r2 = r2_score(y_test, lr_pred)
    return lr_mse, lr_r2

jan_mse, jan_r2 = linreg(df_january)
feb_mse, feb_r2 = linreg(df_february)
mar_mse, mar_r2 = linreg(df_march)
apr_mse, apr_r2 = linreg(df_april)
may_mse, may_r2 = linreg(df_may)
jun_mse, jun_r2 = linreg(df_june)
jul_mse, jul_r2 = linreg(df_july)
aug_mse, aug_r2 = linreg(df_august)
sep_mse, sep_r2 = linreg(df_september)
oct_mse, oct_r2 = linreg(df_october)
nov_mse, nov_r2 = linreg(df_november)
dec_mse, dec_r2 = linreg(df_december)

In [30]:
## monthly results ##
results_monthly = pd.DataFrame(columns=['Month', 'Linear Reg Model', 'Mean Squared Error', 'R-squared'])
results_monthly.loc[0] = ['January', 'classic', jan_mse, jan_r2]
results_monthly.loc[1] = ['February', 'classic', feb_mse, feb_r2]
results_monthly.loc[2] = ['March', 'classic', mar_mse, mar_r2]
results_monthly.loc[3] = ['April', 'classic', apr_mse, apr_r2]
results_monthly.loc[4] = ['May', 'classic', may_mse, may_r2]
results_monthly.loc[5] = ['June', 'classic', jun_mse, jun_r2]
results_monthly.loc[6] = ['July', 'classic', jul_mse, jul_r2]
results_monthly.loc[7] = ['August', 'classic', aug_mse, aug_r2]
results_monthly.loc[8] = ['September', 'classic', sep_mse, sep_r2]
results_monthly.loc[9] = ['October', 'classic', oct_mse, oct_r2]
results_monthly.loc[10] = ['November', 'classic', nov_mse, nov_r2]
results_monthly.loc[11] = ['December', 'classic', dec_mse, dec_r2]
results_monthly = results_monthly.sort_values(by='Mean Squared Error')
results_monthly

,Month,Linear Reg Model,Mean Squared Error,R-squared
2,March,classic,0.053836,0.941260
10,November,classic,0.055033,0.937421
11,December,classic,0.060552,0.938741
3,April,classic,0.062429,0.935754
0,January,classic,0.065798,0.927950
9,October,classic,0.118807,0.888080
1,February,classic,0.139062,0.861714
7,August,classic,0.149058,0.870352
6,July,classic,0.155353,0.850951
4,May,classic,0.239425,0.765497
